In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls '/content/gdrive/My Drive/data_streami

model.h5  sentiment140.csv  tokenizer.pickle


In [0]:
import pandas as pd
import re
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [0]:
print("You have version", tf.__version__)

You have version 1.13.1


In [0]:
#skip the abnormal lines and keeping necessary columns
data_path = '/content/gdrive/My Drive/data_streaming/sentiment140.csv'
data = pd.read_csv(data_path, skiprows=[8835,535881])
data = data[['Sentiment', 'SentimentText']]

In [0]:
from sklearn.utils import shuffle

In [0]:
#data_preprocessing
data['SentimentText'] = data['SentimentText'].apply(lambda x: x.lower())
data['SentimentText'] = data['SentimentText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

positive = data[data['Sentiment'] == 1]
negative = data[data['Sentiment'] == 0]

print(positive.size)
print(negative.size)

data1 = pd.concat([positive.head(20000),negative.head(20000)]) 
data1 = shuffle(data1)


1580354
1576870


In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['SentimentText'].values)
sentiment_text = tokenizer.texts_to_sequences(data1['SentimentText'].values)
sentiment_text = pad_sequences(sentiment_text)
print(type(sentiment_text))

<class 'numpy.ndarray'>


In [0]:
import pickle
with open('/content/gdrive/My Drive/data_streaming/tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [0]:
embedding_size = 16
unit = 8
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_fatures, embedding_size,input_length = sentiment_text.shape[1]))
model.add(tf.keras.layers.LSTM(unit, dropout =0.2, recurrent_dropout =0.2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 16)            32000     
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 800       
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 18        
Total params: 32,818
Trainable params: 32,818
Non-trainable params: 0
_________________________________________________________________


In [0]:
from sklearn.model_selection import train_test_split
sentiment = pd.get_dummies(data1['Sentiment']).values
x_train, x_test, y_train, y_test = train_test_split(sentiment_text, sentiment,test_size=0.2,
                                                    random_state=42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(32000, 40) (32000, 2)
(8000, 40) (8000, 2)


In [0]:
batch_size = 64
model.fit(x_train, y_train, validation_split=0.2,epochs=10, verbose = 2)

Train on 25600 samples, validate on 6400 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 - 65s - loss: 0.6016 - acc: 0.6766 - val_loss: 0.5250 - val_acc: 0.7436
Epoch 2/10
 - 64s - loss: 0.5110 - acc: 0.7561 - val_loss: 0.5165 - val_acc: 0.7498
Epoch 3/10
 - 65s - loss: 0.4909 - acc: 0.7689 - val_loss: 0.5168 - val_acc: 0.7453
Epoch 4/10
 - 64s - loss: 0.4776 - acc: 0.7742 - val_loss: 0.5146 - val_acc: 0.7520
Epoch 5/10
 - 64s - loss: 0.4683 - acc: 0.7836 - val_loss: 0.5153 - val_acc: 0.7525
Epoch 6/10
 - 64s - loss: 0.4626 - acc: 0.7835 - val_loss: 0.5176 - val_acc: 0.7509
Epoch 7/10
 - 64s - loss: 0.4515 - acc: 0.7873 - val_loss: 0.5227 - val_acc: 0.7461
Epoch 8/10
 - 65s - loss: 0.4461 - acc: 0.7915 - val_loss: 0.5286 - val_acc: 0.7525
Epoch 9/10
 - 64s - loss: 0.4411 - acc: 0.7939 - val_loss: 0.5339 - val_acc: 0.7489
Epoch 10/10
 - 64s - loss: 0.4320 - acc: 0.7976 - val_loss: 0.5420 - val_acc: 0.7492


In [0]:
model.evaluate(x_test, y_test, batch_size=32, verbose=1)

8000/8000 [==============================] - 8s 958us/sample - loss: 0.5147 - acc: 0.7595


[0.514716936469078, 0.7595]

In [0]:
from sklearn.externals import joblib

In [0]:
model.save('/content/gdrive/My Drive/data_streaming/model_1.0.h5')

In [0]:
new_model = tf.keras.models.load_model('/content/gdrive/My Drive/data_streaming/model.h5')

W0416 15:39:08.483653 139723704842112 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f1363420518>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0416 15:39:08.547267 139723704842112 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:4081: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#new_model.evaluate(x_test, y_test, batch_size=32, verbose=1)

8000/8000 [==============================] - 12s 1ms/sample - loss: 0.5584 - accuracy: 0.7444


[0.5584347652196884, 0.744375]

In [0]:
import numpy as np

In [0]:
result = np.argmax(new_model.predict(np.array([x_test[0]])))
print(result)

1
